## Решение на 12 место
Хрыльченко Кирилл, Ambitious

В папке utils лежит несколько моих скриптов со всякими штуками, нужными для сабмита:
1. nn.py --- содержит простенькую модель
2. train.py --- cv валидатор
3. evaluate.py --- функции для оптимизации
4. train.py --- класс для обучения модели

P.S: если усреднить больше сидов, то даст и 7-е :D

In [1]:
import numpy as np
import pandas as pd

from collections import defaultdict

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from utils.evaluate import calculate_miou, get_iou, calculate_minmax_loss, get_df_miou
from utils.train import Trainer
from utils.data import load_data

from utils.nn import LinearModel
from utils.train import Validator, HoldoutValidator
from utils.data import Inference, denormalize

torch.manual_seed(0)
np.random.seed(0)

coords = ['Xmin', 'Ymin', 'Xbias', 'Ybias']
GT_coords = ['Xmin_true', 'Ymin_true', 'Xbias_true', 'Ybias_true']

In [2]:
paths = {
    'train': './data/train.csv', # путь к трейн сету
    'test': './data/test.csv', # путь к тест сету
    'targets': './data/targets.csv' # пусть к таргетам
}

#train, test, targets, stats = load_data(paths, normalize=True, bias=True, delete_zero=True)
train, test, targets = load_data(paths, normalize=False, bias=True, delete_zero=True)

In [3]:
class LinearModel(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.cls = nn.Linear(n_features, 4)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        return self.relu(self.cls(x))
    
    def initialize(self, features, regularize=False):
        self.regularize = regularize
        if self.regularize:
            self.reg_positions = []
        n = len(features)
        weight = []
        for idx, word in enumerate(['Xmin_min', 'Ymin_min', 'Xbias_opt', 'Ybias_opt']):
            pos = features.index(word)
            if self.regularize:
                self.reg_positions.append(n * idx + pos)
            weight.append([0] * pos + [1] + [0] * (n - pos - 1))
        self.cls.weight.data = torch.tensor(weight).float()
        self.cls.bias.data = torch.zeros(4).float()
        
    def L1_loss(self, coef=1.):
        flattened_weights = self.cls.weight.flatten()
        
        term_1 = coef * (1. - flattened_weights[self.reg_positions]).abs().sum()
        
        term_2 = flattened_weights[:self.reg_positions[0]].abs().sum()
        for i in range(len(self.reg_positions) - 1):
            term_2 += flattened_weights[self.reg_positions[i] + 1 : self.reg_positions[i + 1]].abs().sum()
        term_2 += flattened_weights[self.reg_positions[i + 1] + 1: ].abs().sum() 
        term_2 += self.cls.bias.flatten().abs().sum()
        term_2 *= coef 
        return term_1 + term_2
    
    def L2_loss(self, coef=1.):
        flattened_weights = self.cls.weight.flatten()
        
        term_1 = coef * (1. - flattened_weights[self.reg_positions]).pow(2).sum()
        
        term_2 = flattened_weights[:self.reg_positions[0]].pow(2).sum()
        for i in range(len(self.reg_positions) - 1):
            term_2 += flattened_weights[self.reg_positions[i] + 1 : self.reg_positions[i + 1]].pow(2).sum()
        term_2 += flattened_weights[self.reg_positions[i + 1] + 1: ].pow(2).sum()
        term_2 += self.cls.bias.flatten().pow(2).sum()
        term_2 *= coef / 2.
        return term_1 + term_2

In [4]:
class ItemDataset(Dataset):
    def __init__(self, df, targets=None):
        df = df.copy()
        for coord in ['X', 'Y']:
            df['{}max'.format(coord)] = df['{}min'.format(coord)] + df['{}bias'.format(coord)]
            
        df['square'] = np.sqrt(df['Xbias'] * df['Ybias'])
        data = df.groupby('item').agg({
            'Xmin': ['min'],
            'Ymin': ['min'],
            'Xmax': ['max'],
            'Ymax': ['max'],
            'square': ['mean']
        })
        
        data.columns = ['{}_{}'.format(first, second) for first, second in data.columns]
        for coord in ['X', 'Y']:
            data['{}bias_opt'.format(coord)] = data['{}max_max'.format(coord)] - data['{}min_min'.format(coord)]
        data.drop(['Xmax_max', 'Ymax_max'], axis=1, inplace=True)
        
        for col in ['Xbias_opt', 'Ybias_opt', 'Xmin_min']:
            data['{}_sq]'.format(col)] = data[col] ** 2 / 1000.

        self.features = list(data.columns)
        self.items = list(data.index)
        self.data = data.values.astype(np.float32)
        self.len = self.data.shape[0]
        self.n_features = self.data.shape[1]
        
        self.targets = None
        if targets is not None:
            self.targets = targets.set_index('item').loc[self.items].values.astype(np.float32)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        
        if self.targets is not None:
            return self.data[idx], self.targets[idx]
        else:
            return self.data[idx]

In [5]:
def make_dataloader(df, targets):
    ds = ItemDataset(df, targets)
    return DataLoader(ds, batch_size=256)

def make_model(ds):
    model = LinearModel(ds.n_features)
    model.initialize(ds.features, regularize=True)
    return model

def make_criterion(model):
    criterion = lambda preds, targets: calculate_miou(preds, targets) + model.L2_loss(0.001) + \
        0.000001 * torch.pow(preds - targets, 2).sum(dim=1).mean(dim=0) / torch.pow(targets, 2).sum(dim=1).mean(dim=0)
    return criterion

def make_val_criterion(model):
    val_criterion = lambda preds, targets: calculate_miou(preds, targets)
    return val_criterion

train_params = {
    'n_epochs': 1000,
    'patience': 80,
    'n_restarts': 1,
    'gamma': 0.05,
    'verbose': False
}

In [6]:
all_models = []

for seed in range(10):
    val = Validator(train, targets, n_splits=5, seed=seed)

    models, losses = val.train(
        make_dataloader, make_model, make_criterion, make_val_criterion, train_params, verbose=True
    )
    
    print()
    all_models += models

fold:  1, loss: 0.5927, epoch:  272, time: 10.521s
fold:  2, loss: 0.5968, epoch:  562, time: 14.811s
fold:  3, loss: 0.5935, epoch:  351, time: 8.179s
fold:  4, loss: 0.5867, epoch:  258, time: 6.828s
fold:  5, loss: 0.6009, epoch:  357, time: 6.467s
mean: 0.5941, std: 0.0047

fold:  1, loss: 0.6027, epoch:  301, time: 6.470s
fold:  2, loss: 0.5572, epoch:  522, time: 8.342s
fold:  3, loss: 0.5940, epoch:  321, time: 6.465s
fold:  4, loss: 0.6168, epoch:  514, time: 9.160s
fold:  5, loss: 0.5960, epoch:  226, time: 4.321s
mean: 0.5933, std: 0.0198

fold:  1, loss: 0.5822, epoch:  316, time: 5.360s
fold:  2, loss: 0.6009, epoch:  272, time: 5.468s
fold:  3, loss: 0.5993, epoch:  349, time: 6.638s
fold:  4, loss: 0.5891, epoch:  394, time: 7.127s
fold:  5, loss: 0.5959, epoch:  519, time: 8.619s
mean: 0.5935, std: 0.0070

fold:  1, loss: 0.5842, epoch:  474, time: 7.931s
fold:  2, loss: 0.6070, epoch:  223, time: 4.830s
fold:  3, loss: 0.5934, epoch:  574, time: 9.500s
fold:  4, loss: 0

In [7]:
# 0.5773556 public

inf = Inference(lambda df: ItemDataset(df))
test_preds = inf.predict(test, all_models)

#denormalize(test_preds, stats).to_csv('test_preds.csv', index=False, header=False)
np.round(test_preds).to_csv('test_preds_10.csv', index=False, header=False)

In [9]:
np.round(test_preds).head(n=20)

,itemId,Xmin,Ymin,Xmax,Ymax
0,18,107.0,629.0,756.0,1117.0
1,19,27.0,559.0,138.0,704.0
2,33,20.0,378.0,412.0,653.0
3,62,35.0,822.0,631.0,1256.0
4,114,38.0,569.0,231.0,757.0
5,146,0.0,213.0,783.0,855.0
6,156,67.0,801.0,159.0,884.0
7,163,19.0,316.0,216.0,462.0
8,164,0.0,145.0,469.0,576.0
9,179,98.0,488.0,336.0,714.0
